# BASELINE

------------------------------------------

## Common Utils

In [ ]:
import sys
from metrics import *

sys.path.append('data')

from data.constants import *
from constants import *

## Optuna

In [ ]:
from enum import Enum

class ChannelAllowed(Enum):
    RGB = RGB_CHANNEL,
    NDVI = NVDI_CHANNEL

In [ ]:
import optuna
import torchvision.transforms as transforms

from PIL import Image
from data.processing import convert_to_black_and_white


def objective(trial, images, ground_truth, channel: ChannelAllowed):

    # Define the search space for hyperparameters
    threshold  = trial.suggest_int('threshold ', 0, 255)
    transform = transforms.ToTensor()

    mean_iou = 0

    for image_path, ground_truth_path in zip(images, ground_truth):
        
        image = Image.open(image_path)      
        
        gt = Image.open(ground_truth_path)
        gt = convert_to_black_and_white(image=gt, save_results=False, threshold=1)
        gt = transform(gt).squeeze(0)

        width, height = image.size
        
        pixels = list(image.getdata())
        vegetation = []
        
        for pixel in pixels:
            
            if channel == ChannelAllowed.RGB:
                vegetation_index = calculate_exg_index(pixel)
            else:
                # NDVI
                vegetation_index = pixel
        
            if vegetation_index > threshold:
                vegetation.append(1)
            else:
                vegetation.append(0)

        vegetation = torch.tensor(vegetation)
        vegetation = vegetation.view(height, width)
        
        # Calculate the mean IoU
        mean_iou += calculate_miou(vegetation, gt, 2)
        # print(f"Mean IoU: {mean_iou/len(images)}")
        
    return mean_iou / len(images)
            

### Baseline

In [ ]:
import os
import optuna
import numpy as np
import tqdm
from data.constants import *
from data.utils import load_data_path

dir_path = f"{OPTUNA}/{BASELINE}"
data = load_data_path()
study_channel = ChannelAllowed.NDVI.value

#### Separate images

In [ ]:
storage_name = f"sqlite:///baseline_{study_channel}_separate_images.db"

In [ ]:
for drone, drone_dir in data.items():
    print(f"Drone: {drone}")
    for image, image_dir in drone_dir.items():
        
        tiles = image_dir[TILE][study_channel]
        ground_truth = image_dir[GROUND_TRUTH]
        pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
        
        study = optuna.create_study(study_name=image, direction='maximize', storage=storage_name, pruner=pruner, load_if_exists=True)
        study.optimize(lambda trial: objective(trial, tiles, ground_truth, study_channel), n_trials=20)

        # Get the best hyperparameters
        try:
            best_params = study.best_params
            print(f"Best Hyperparameters {image}:", best_params)
        except ValueError:
            print(f"No best hyperparameters found for {key}")

#### All images togheter

In [ ]:
storage_name = f"sqlite:///baseline_{study_channel}_all.db"

In [ ]:
# Merge images from different drones

tiles = []
ground_truth = []
images = []

for drone, drone_dir in data.items():
    for image, image_dir in drone_dir.items():     
        images += image   
        tiles += image_dir[TILE][study_channel]
        ground_truth += image_dir[GROUND_TRUTH]

study_name = 'all'
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)

study = optuna.create_study(study_name=study_name, direction='maximize', storage=storage_name, pruner=pruner, load_if_exists=True)
study.optimize(lambda trial: objective(trial, tiles, ground_truth, study_channel), n_trials=0) 
# Get the best hyperparameters
try:
    best_params = study.best_params
    print(f"Best Hyperparameters:", best_params)
except ValueError:
    print(f"No best hyperparameters found")

#### Test

In [ ]:
storage_name = f"sqlite:///{dir_path}/{study_channel}_test.db"

In [ ]:
# Merge images from different drones

tiles = []
ground_truth = []
images = []

for drone, drone_dir in data.items():
    for image, image_dir in drone_dir.items():   
        if image in ('003', '005'):
            images += image   
            tiles += image_dir[TILE][study_channel]
            ground_truth += image_dir[GROUND_TRUTH]

study_name = 'test'
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)

study = optuna.create_study(study_name=study_name, direction='maximize', storage=storage_name, pruner=pruner, load_if_exists=True)
study.optimize(lambda trial: objective(trial, tiles, ground_truth, study_channel), n_trials=20) 
# Get the best hyperparameters
try:
    best_params = study.best_params
    print(f"Best Hyperparameters:", best_params)
except ValueError:
    print(f"No best hyperparameters found")

In [ ]:
from comet_ml import Experiment

experiment = Experiment(
    api_key="eI2MJOa5W8d1PcAvxhmyP5VGt",
    project_name="weedmap-image-segmentation",
    workspace="francesco-ranieri"
)

experiment.set_name('baseline_ndvi')
experiment.add_tags(['baseline', 'ndvi', 'test'])
experiment.log_parameters(best_params)
experiment.log_metric('mIOU', study.best_value)
experiment.end()

#### Train

In [ ]:
storage_name = f"sqlite:///{dir_path}/{study_channel}_train.db"

In [ ]:
# Merge images from different drones

tiles = []
ground_truth = []
images = []

for drone, drone_dir in data.items():
    for image, image_dir in drone_dir.items():   
        if image not in ('003', '005'):
            images += image   
            tiles += image_dir[TILE][study_channel]
            ground_truth += image_dir[GROUND_TRUTH]

study_name = 'train'
pruner = optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)

study = optuna.create_study(study_name=study_name, direction='maximize', storage=storage_name, pruner=pruner, load_if_exists=True)
study.optimize(lambda trial: objective(trial, tiles, ground_truth, study_channel), n_trials=20) 
# Get the best hyperparameters
try:
    best_params = study.best_params
    print(f"Best Hyperparameters:", best_params)
except ValueError:
    print(f"No best hyperparameters found")

In [ ]:
from comet_ml import Experiment

experiment = Experiment(
    api_key="eI2MJOa5W8d1PcAvxhmyP5VGt",
    project_name="weedmap-image-segmentation",
    workspace="francesco-ranieri"
)

experiment.set_name('baseline_train_ndvi')
experiment.add_tags(['baseline', 'ndvi', 'train'])
experiment.log_parameters(best_params)
experiment.log_metric('mIOU', study.best_value)
experiment.end()